In [1]:
import numpy as np
import numpy_linalg as la
import complex_synapse as cs
from sl_py_tools.import_tricks import reload

In [2]:
np.set_printoptions(precision=4, suppress=True)

In [89]:
reload(cs)

Reloading complex_synapse.synapse_memory_model
Reloading complex_synapse.synapse_base
Reloading complex_synapse.builders
Reloading complex_synapse.synapse_opt


In [3]:
def balance_grad(model: cs.SynapseMemoryModel, s: float) -> float:
    W = model.markov()
    Zi = model.zinv(s)
    K = model.enc()
    p = model.peq()
    Zid = cs.builders.adjoint(Zi, p)
    return (p.r @ K @ Zi.inv @ (la.eye(model.nstates) - Zid @ Zi.inv) @ model.weight.c).us

In [4]:
s = 4
n = 6

In [5]:
syn = cs.SynapseMemoryModel.rand(n, binary=True)
ss = la.linspace(0, s, 10)
print('gradient', balance_grad(syn, ss))
print('laplace', syn.snr_laplace(ss))
print('sign', (balance_grad(syn, ss) * syn.snr_laplace(ss) >= 0).all())

gradient [0.0494 0.0349 0.0258 0.0198 0.0155 0.0125 0.0102 0.0085 0.0072 0.0061]
laplace [0.2677 0.2306 0.2026 0.1807 0.163  0.1485 0.1363 0.126  0.1172 0.1095]
sign True


In [97]:
W = syn.markov()
Zi = syn.zinv(s)
K = syn.enc()
p = syn.peq()
Zid = cs.builders.adjoint(Zi, p)